# 🛰️ EO-AI Portfolio: Quick Demo

**Land Cover Classification from Sentinel-2 Imagery**

This notebook demonstrates:
1. Data loading and preprocessing
2. Model inference
3. Visualization of results
4. Performance metrics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yourusername/EO-AI-Portfolio/blob/main/notebooks/01_quick_demo.ipynb)

## 📦 Setup

In [ ]:
# Install dependencies (uncomment for Colab)
# !pip install torch torchvision albumentations matplotlib seaborn

import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 🌍 Generate Sample Data

In [ ]:
# Generate a small synthetic dataset
!python ../data/download_bigearthnet_subset.py --output ../data/demo --num_samples 100

print("✅ Dataset created!")

## 📊 Visualize Sample Data

In [ ]:
from data.preprocess import Sentinel2Dataset, get_transforms
from PIL import Image

# Load dataset
dataset = Sentinel2Dataset(
    '../data/demo',
    split='train',
    transform=get_transforms('val')  # No augmentation for visualization
)

# Visualize first 4 samples
fig, axes = plt.subplots(4, 3, figsize=(12, 16))

CLASS_COLORS = np.array([
    [230, 0, 0], [180, 0, 0], [255, 255, 0], [240, 150, 0],
    [150, 255, 0], [200, 200, 0], [0, 150, 0], [150, 200, 150],
    [200, 200, 200], [0, 100, 200]
], dtype=np.uint8)

def colorize_mask(mask):
    h, w = mask.shape
    colored = np.zeros((h, w, 3), dtype=np.uint8)
    for i in range(10):
        colored[mask == i] = CLASS_COLORS[i]
    return colored

for i in range(4):
    image, mask = dataset[i]
    
    # Convert to numpy
    image_np = image.numpy().transpose(1, 2, 0)  # (4, H, W) -> (H, W, 4)
    mask_np = mask.numpy()
    
    # RGB composite
    rgb = image_np[:, :, :3]
    rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min())
    
    # NIR band
    nir = image_np[:, :, 3]
    
    # Colored mask
    colored_mask = colorize_mask(mask_np)
    
    # Plot
    axes[i, 0].imshow(rgb)
    axes[i, 0].set_title(f'Sample {i+1}: RGB')
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(nir, cmap='gray')
    axes[i, 1].set_title('NIR Band')
    axes[i, 1].axis('off')
    
    axes[i, 2].imshow(colored_mask)
    axes[i, 2].set_title('Land Cover Mask')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

print(f"Dataset size: {len(dataset)} samples")

## 🏋️ Train a Quick Model (Optional)

In [ ]:
# Quick training for 5 epochs
!python ../train.py \
    --data_path ../data/demo \
    --epochs 5 \
    --batch_size 4 \
    --lr 1e-3 \
    --checkpoint_dir ../checkpoints_demo

## 🔮 Run Inference

In [ ]:
from models.unet import get_model
from inference import predict_single_image, colorize_mask
import time

# Load model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_model(n_channels=4, n_classes=10)

# Load checkpoint (if available)
checkpoint_path = Path('../checkpoints_demo/best_model.pth')
if checkpoint_path.exists():
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print("✅ Model loaded!")
else:
    print("⚠️ No checkpoint found, using random weights")

model = model.to(device)
model.eval()

print(f"Device: {device}")

In [ ]:
# Run inference on test samples
test_dataset = Sentinel2Dataset(
    '../data/demo',
    split='test',
    transform=get_transforms('val')
)

fig, axes = plt.subplots(3, 3, figsize=(15, 15))

for i in range(3):
    image_tensor, mask_gt = test_dataset[i]
    image_np = image_tensor.numpy().transpose(1, 2, 0)
    
    # Predict
    with torch.no_grad():
        image_input = image_tensor.unsqueeze(0).to(device)
        start = time.time()
        output = model(image_input)
        inference_time = (time.time() - start) * 1000
        
        prediction = output.argmax(dim=1).squeeze(0).cpu().numpy()
    
    # Visualize
    rgb = image_np[:, :, :3]
    rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min())
    
    axes[i, 0].imshow(rgb)
    axes[i, 0].set_title(f'Input RGB')
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(colorize_mask(prediction))
    axes[i, 1].set_title(f'Prediction ({inference_time:.1f}ms)')
    axes[i, 1].axis('off')
    
    axes[i, 2].imshow(colorize_mask(mask_gt.numpy()))
    axes[i, 2].set_title('Ground Truth')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

## 📊 Calculate Metrics

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Collect predictions
all_preds = []
all_targets = []

with torch.no_grad():
    for i in range(len(test_dataset)):
        image, mask = test_dataset[i]
        image = image.unsqueeze(0).to(device)
        
        output = model(image)
        pred = output.argmax(dim=1).squeeze(0).cpu().numpy()
        
        all_preds.append(pred.flatten())
        all_targets.append(mask.numpy().flatten())

all_preds = np.concatenate(all_preds)
all_targets = np.concatenate(all_targets)

# Calculate mIoU
ious = []
for cls in range(10):
    pred_mask = all_preds == cls
    target_mask = all_targets == cls
    
    intersection = (pred_mask & target_mask).sum()
    union = (pred_mask | target_mask).sum()
    
    if union > 0:
        ious.append(intersection / union)

miou = np.mean(ious)
accuracy = (all_preds == all_targets).sum() / len(all_targets)

print(f"\n📊 Metrics:")
print(f"  mIoU: {miou:.4f}")
print(f"  Accuracy: {accuracy:.4f}")

# Confusion matrix
cm = confusion_matrix(all_targets, all_preds)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix (Normalized)')
plt.tight_layout()
plt.show()

## 🚀 Model Compression Demo

In [ ]:
# Benchmark original model
import time

def benchmark_model(model, input_shape=(1, 4, 256, 256), num_runs=100):
    model.eval()
    dummy_input = torch.randn(input_shape).to(device)
    
    # Warmup
    with torch.no_grad():
        for _ in range(10):
            _ = model(dummy_input)
    
    # Benchmark
    times = []
    with torch.no_grad():
        for _ in range(num_runs):
            start = time.time()
            _ = model(dummy_input)
            if device.type == 'cuda':
                torch.cuda.synchronize()
            times.append(time.time() - start)
    
    avg_time = np.mean(times) * 1000  # ms
    return avg_time

original_time = benchmark_model(model)

print(f"⏱️ Original model inference: {original_time:.2f} ms")
print(f"🚀 FPS: {1000/original_time:.2f}")

# Note: Full quantization requires CPU and calibration data
# See models/quantization.py for complete pipeline

## 🎯 Summary

This notebook demonstrated:
- ✅ Synthetic Sentinel-2 data generation
- ✅ U-Net model for land cover classification
- ✅ Inference and visualization
- ✅ Performance metrics (mIoU, accuracy)

### Next Steps:
1. Train on real BigEarthNet data
2. Apply INT8 quantization for 3x compression
3. Deploy with Docker
4. Scale with multi-GPU DDP training

### Links:
- 📖 [README](../README.md)
- 🐙 [GitHub Repository](https://github.com/yourusername/EO-AI-Portfolio)
- 📄 [Full Documentation](../README.md#documentation)